In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)

Store Part I results into DataFrame

In [2]:
data = pd.read_csv('../Cities_Weather_Info.csv')
data = data.dropna()
data

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,City Date/Time
0,0,narsaq,60.9167,-46.0500,9.12,74.0,99.0,1.37,GL,1.656135e+09
1,1,namatanai,-3.6667,152.4333,29.24,69.0,63.0,5.57,PG,1.656135e+09
2,2,vaini,-21.2000,-175.2000,23.09,73.0,20.0,4.12,TO,1.656135e+09
4,4,lompoc,34.6391,-120.4579,18.08,95.0,0.0,3.09,US,1.656135e+09
5,5,bluff,-46.6000,168.3333,9.89,90.0,99.0,3.33,NZ,1.656135e+09
...,...,...,...,...,...,...,...,...,...,...
571,571,pahrump,36.2083,-115.9839,30.14,16.0,0.0,0.89,US,1.656135e+09
572,572,trairi,-3.2778,-39.2689,22.12,95.0,44.0,2.62,BR,1.656135e+09
573,573,loandjili,-4.7561,11.8578,22.06,94.0,20.0,0.51,CG,1.656135e+09
574,574,coihaique,-45.5752,-72.0662,-7.03,86.0,0.0,1.03,CL,1.656135e+09


Humidity Heatmap

In [3]:
#store latitude and longitude as locations
locations = data[["Latitude", "Longitude"]]
humidity = data["Humidity"]

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}


fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False,max_intensity=max(data["Humidity"]),
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

Create new DataFrame fitting weather criteria

In [4]:
ideal_dataframe = data.loc[data['Humidity'] > 30]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Humidity'] < 70]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Max Temp'] < 32]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Max Temp'] > 28]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Cloudiness'] < 30]
ideal_dataframe

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,City Date/Time
58,58,uri,34.0833,74.0333,29.34,35.0,0.0,1.70,IN,1.656135e+09
177,177,ishinomaki,38.4167,141.3000,31.00,66.0,9.0,4.75,JP,1.656135e+09
195,195,lugovoy,42.9472,72.7644,30.86,33.0,0.0,4.01,KZ,1.656135e+09
306,306,dingle,10.9995,122.6711,31.15,65.0,19.0,1.16,PH,1.656135e+09
434,434,laguna,38.4210,-121.4238,28.55,51.0,0.0,3.09,US,1.656135e+09
440,440,sivaki,52.6333,126.7500,28.75,31.0,10.0,4.88,RU,1.656135e+09


Hotel Map

In [5]:
hotel_df = ideal_dataframe
hotel_df["Hotel Name"] = ""

params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():

    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City Name"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    hotel_name = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    hotel_name = hotel_name.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print(f"Could not find a hotel in {city}")

# Visualize to confirm lat lng appear
hotel_df

Could not find a hotel in uri
Could not find a hotel in ishinomaki
Could not find a hotel in lugovoy
Could not find a hotel in dingle
Could not find a hotel in laguna
Could not find a hotel in sivaki


,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,City Date/Time,Hotel Name
58,58,uri,34.0833,74.0333,29.34,35.0,0.0,1.70,IN,1.656135e+09,
177,177,ishinomaki,38.4167,141.3000,31.00,66.0,9.0,4.75,JP,1.656135e+09,
195,195,lugovoy,42.9472,72.7644,30.86,33.0,0.0,4.01,KZ,1.656135e+09,
306,306,dingle,10.9995,122.6711,31.15,65.0,19.0,1.16,PH,1.656135e+09,
434,434,laguna,38.4210,-121.4238,28.55,51.0,0.0,3.09,US,1.656135e+09,
440,440,sivaki,52.6333,126.7500,28.75,31.0,10.0,4.88,RU,1.656135e+09,


In [6]:
hotel_df = hotel_df.rename(columns = {"City Name": "City"})

In [7]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…